In [ ]:
import pandas as pd
import json

# Load the Excel file
data = pd.read_excel('grouped_data.xlsx')

In [ ]:
# Function to clean and normalize the "Barrage content"
def clean_barrage_content(content):
    if pd.isna(content):
        return []

    try:
        # Convert the string representation of the list to an actual list
        content_list = eval(content)

        # Initialize an empty list to store the cleaned data
        cleaned_data = []

        for item in content_list:
            # Some entries might be empty strings or malformed JSON
            if item:
                try:
                    json_data = json.loads(item)
                    cleaned_data.extend(json_data)
                except json.JSONDecodeError:
                    continue

        return cleaned_data
    except:
        return []

In [3]:
# Apply the function to clean the "Barrage content" column
data['Cleaned Barrage Content'] = data['Barrage content'].apply(clean_barrage_content)

# Explode the cleaned data into separate rows for each word-count pair
exploded_data = data.explode('Cleaned Barrage Content').reset_index(drop=True)

# Normalize the 'Cleaned Barrage Content' dictionaries into separate columns
normalized_data = pd.json_normalize(exploded_data['Cleaned Barrage Content'])

# Combine the normalized data with the original dataframe
final_data = pd.concat([exploded_data.drop(columns=['Cleaned Barrage Content']), normalized_data], axis=1)

# Save the final structured data to a new Excel file
final_data.to_excel('cleaned_grouped_data.xlsx', index=False)
